# NLP - lab2

### Mateusz Praski

---


In [1]:
import requests
import json

import pandas as pd

from tqdm import trange

In [2]:
analyzer_with_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'months-synonyms',
        'lowercase',
        'morfologik_stem',
        'lowercase'
    ]
}

analyzer_without_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'lowercase',
        'morfologik_stem',
        'lowercase'
    ]
}

filters = {
    'months-synonyms': {
        'type': 'synonym',
        'synonyms': [
            'sty, I => styczeń',
            'lut, II => luty',
            'mar, III => marzec',
            'kwi, IV => kwiecień',
            'V => maj',
            'cze, VI => czerwiec',
            'lip, VII => lipca',
            'sie, VIII => sierpnia',
            'wrz, IX => wrzesień',
            'paz, X => pażdziernik',
            'lis, XI => listopad',
            'gru, XII => grudzień'
        ]
    }
}

In [3]:
index_definition = {
    'mappings': {
        'properties': {
            'with_synonyms': {
                'type': 'text',
                'analyzer': 'analyze_with_synonyms'
            },
            'without_synonyms': {
                'type': 'text',
                'analyzer': 'analyze_without_synonyms'
            }
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'analyze_with_synonyms': analyzer_with_synonyms,
                'analyze_without_synonyms': analyzer_without_synonyms
            },
            'filter': filters
        }
    }
}

In [4]:
body = json.dumps(index_definition, indent=4)
print(body)

{
    "mappings": {
        "properties": {
            "with_synonyms": {
                "type": "text",
                "analyzer": "analyze_with_synonyms"
            },
            "without_synonyms": {
                "type": "text",
                "analyzer": "analyze_without_synonyms"
            }
        }
    },
    "settings": {
        "analysis": {
            "analyzer": {
                "analyze_with_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "months-synonyms",
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyze_without_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
         

In [5]:
!docker cp elastic_container:/usr/share/elasticsearch/config/certs/http_ca.crt .

                    'c.
                 ,xNMM.
               .OMMMMo
               OMMM0,
     .;loddo:' loolloddol;.
   cKMMMMMMMMMMNWMMMMMMMMMM0:
 .KMMMMMMMMMMMMMMMMMMMMMMMWd.
 XMMMMMMMMMMMMMMMMMMMMMMMX.
;MMMMMMMMMMMMMMMMMMMMMMMM:
:MMMMMMMMMMMMMMMMMMMMMMMM:
.MMMMMMMMMMMMMMMMMMMMMMMMX.
 kMMMMMMMMMMMMMMMMMMMMMMMMWd.
 .XMMMMMMMMMMMMMMMMMMMMMMMMMMk
  .XMMMMMMMMMMMMMMMMMMMMMMMMK.
    kMMMMMMMMMMMMMMMMMMMMMMd
     ;KMMMMMMMWXXWMMMMMMMk.
       .cooc,.    .,coo:.
thmtt@Mateuszs-MacBook-Air-2.local 
---------------------------------- 
OS: macOS 14.0 23A344 arm64 
Host: MacBookAir10,1 
Kernel: 23.0.0 
Uptime: 18 days, 23 hours, 1 min 
Packages: 164 (brew) 
Shell: fish 3.4.1 
Resolution: 1440x900 
DE: Aqua 
WM: Rectangle 
Terminal: python 
CPU: Apple M1 
GPU: Apple M1 
Memory: 2777MiB / 16384MiB 

                        
                        




In [6]:
ELASTIC_IP = 'https://localhost:9200'
INDEX = 'nlp-index'
auth = ('elastic', 'qwerty')
cert = 'http_ca.crt'

In [7]:
res = requests.delete(f'{ELASTIC_IP}/{INDEX}', auth=auth, verify=cert)
res.status_code, print(res.content.decode())

{"acknowledged":true}


(200, None)

In [8]:
res = requests.put(f'{ELASTIC_IP}/nlp-index', json=index_definition, auth=('elastic', 'qwerty'), verify='http_ca.crt')
res.status_code, print(res.content.decode())

{"acknowledged":true,"shards_acknowledged":true,"index":"nlp-index"}


(200, None)

In [9]:
df = pd.read_json("../../data/corpus.jsonl", lines=True)
df = df.set_index('_id').sort_index()
df.head()

,title,text,metadata
_id,,,
3,,"Nie mówię, że nie podoba mi się też pomysł szk...",{}
31,,Tak więc nic nie zapobiega fałszywym ocenom po...,{}
56,,Nigdy nie możesz korzystać z FSA dla indywidua...,{}
59,,Samsung stworzył LCD i inne technologie płaski...,{}
63,,Oto wymagania SEC: Federalne przepisy dotycząc...,{}


In [10]:
corpus = [i for i in df['text']]
corpus[:2]

['Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.',
 'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO jest śmieciem, czy nie. Edycja: wprowadzenie']

In [38]:
def bulk_insert(rows, col):
    n = len(rows)
    request_command = [f'{{"index": {{ "_index": "{INDEX}" }} }}'] * n
    request_data = [f'{{"with_synonyms": {text}, "without_synonyms": {text}}}' for text in rows]

    payload = [None] * (n * 2)
    payload[::2] = request_command
    payload[1::2] = request_data
    body = "\n".join(payload) + '\n'
    print(body)
    res = requests.post(f'{ELASTIC_IP}/_bulk?pretty', data=body, headers={'Content-type': 'application/x-ndjson'}, auth=auth, verify=cert)
    return res

In [39]:
res = bulk_insert(corpus[:2], "with_synonyms")

{"index": { "_index": "nlp-index" } }
{"with_synonyms": Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani., "without_synonyms": Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikow

In [40]:
res.status_code

400

In [41]:
print(res.content.decode())

{
  "error" : {
    "root_cause" : [
      {
        "type" : "illegal_argument_exception",
        "reason" : "The bulk request must be terminated by a newline [\\n]"
      }
    ],
    "type" : "illegal_argument_exception",
    "reason" : "The bulk request must be terminated by a newline [\\n]"
  },
  "status" : 400
}



In [42]:
text = corpus[0]
text

'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'

In [49]:
res = requests.post(f"{ELASTIC_IP}/{INDEX}/_doc", json={"with_synonyms": text, "without_synonyms": text}, auth=auth, verify=cert)

In [50]:
res.status_code

201

In [51]:
res.content

b'{"_index":"nlp-index","_id":"HSUCV4sBWyMtNHwHLHa7","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}'

In [57]:
res = requests.get(f"{ELASTIC_IP}/{INDEX}/_search?pretty", auth=auth, verify=cert, json={
    "query": {
        "match": {
            "with_synonyms": {
                "query": "pracownik"
            }
        }
    }
})

In [58]:
res.status_code

200

In [60]:
print(res.content.decode('utf-8'))

{
  "took" : 17,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 1,
      "relation" : "eq"
    },
    "max_score" : 0.3288667,
    "hits" : [
      {
        "_index" : "nlp-index",
        "_id" : "HSUCV4sBWyMtNHwHLHa7",
        "_score" : 0.3288667,
        "_source" : {
          "with_synonyms" : "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.",
          "without_synonyms" : "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie może